#### THINGS TO ADD


#### SCS Implementation

This section outlines the symbol clock synchronization (SCS) implementation, highlighting the various submodules needed to correct symbol clock offsets and explaining their functionalities. SCS is essential in communication systems with non-coherent transmitter and receiver architectures, which can lead to a lack of synchronization between the two. While a phase-locked loop (PLL) can track the envelope of the received signal, the analog-to-digital converter (ADC) samples the signal based on its own internal clock, which is unaware of the optimal symbol sample times. The SCS addresses the small clock offsets resulting from this desynchronization, enabling sampling at the ideal symbol time and thereby reducing the likelihood of symbol errors. Figure 1 below displays the different SCS submodules and their placement in the overall system.

SCS SUBMODULE DIAGRAM
<!-- <div style="text-align: center;">
    <img src="./images/QPSK/transmitter_receiver_diagram.png" alt="" width="750" />
    <p>QPSK System Transmitter and Receiver Architecture</p>
</div> -->

#### Imports and Library Functions

talk about library and imports here

In [1]:
from helper_functions import sp_library as sp
import numpy as np 
import matplotlib.pyplot as plt

#### Timing Error Detector
As seen in the diagram above, the timing error detector (TED), receives an input signal at 2 samples per symbol. The general foundation of the TED is that it looks at the samples on either side of the current sample and uses the slope between the two, along with the sign of the current sample to calculate a proportional clock adjustment to be made. This is shown in Figure X below.

TED SAMPLE ERROR CALC DIAGRAM

$$
e\left(nT\right)=x\left(nT_s+\tau \right)\left(x\left(\left(n+1\right)T_s+\tau \right)-x\left(\left(n-1\right)T_s+\tau \right)\right)
$$

where:
- $x(nT_s)$ is the current sample,
- $x((n+1)T_s)$ is the next sample (often referred to as late),
- $x((n-1)T_s)$ is the previous sample (often referred to as early),
- $\tau$ is the previously calculated clock offset.

This calculation can be made for the real or imaginary channels in a quadrature system as it is assumed that the same ADC was used for discretization and therefore the two channels are synchronized. The following excerpt shoes the timing error detector implementation used later in the full system.

In [3]:
def early_late_ted(early_sample, current_sample, late_sample, gain):
        e_nT = (late_sample - early_sample) * (-1 if current_sample < 0 else 1)
        return e_nT * gain

# test samples
early_sample = 1.0
current_sample = -0.5
late_sample = 2.0
gain = 2

ted_output = early_late_ted(early_sample, current_sample, late_sample, gain)
print(f"Calculated Clock Offset): {ted_output}")

Calculated Clock Offset): -2.0


#### Loop Filter
The loop filter module provides stability for the overall SCS sstem by shaping the closed-loop frequency (transient) response. During instantiation of the SCS system, a loop bandwidth and damping factor are defined as parameters shaping this transient response. The loop bandwidth specifies the speed at which the SCS will converge towards zeroing the received symbol clock offset. Setting a wider loop bandwidth allows the SCS to respond more rapidly respond to different clock offsets but introduces more internal noise. The damping factor, often represented as $\zeta$ specifies how the oscillations decay in the transient response when a input frequency change is introduced. Together these parameters categorize the loop filter coefficients $K_1$ and $K_2$ which are derived below.

LOOP FILTER DIAGRAM HERE

$$
K_1 = \frac{4 \xi \left( \frac{B_n T_s}{\zeta + \frac{1}{4 \zeta}} \right)}{1 + 2 \zeta \left( \frac{B_{nT} T_s}{\zeta + \frac{1}{4 \zeta}} \right) + \left( \frac{B_{nT} T_s}{\zeta + \frac{1}{4 \zeta}} \right)^2} \quad \quad \quad K_2 = \frac{4 \left( \frac{B_n T_s}{\zeta + \frac{1}{4 \zeta}} \right)^2}{1 + 2 \zeta \left( \frac{B_{nT} T_s}{\zeta + \frac{1}{4 \zeta}} \right) + \left( \frac{B_{nT} T_s}{\zeta + \frac{1}{4 \zeta}} \right)^2}
$$

where:
- $B_n$ represents the loop bandwidth (usually normalized for the sample rate $f_s$),
- $\zeta$ represents the damping factor,
- $T_s$ is the sampling period of the system.

In [4]:
def compute_loop_constants(fs, lb, df):
    denominator = 1 + ((2 * df) * ((lb * (1 / fs)) / (df + (1 / (4 * df))))) + ((lb * (1 / fs)) / (df + (1 / (4 * df)))) ** 2
    K1 = ((4 * df) * ((lb * (1 / fs)) / (df + (1 / (4 * df))))) / denominator
    K2 = (((lb * (1 / fs)) / (df + (1 / (4 * df)))) ** 2) / denominator
    return K1, K2

sample_rate = 8
loop_bandwidth = 0.02 * sample_rate
damping_factor = 1 / np.sqrt(2)
k1, k2 = compute_loop_constants(sample_rate, loop_bandwidth, damping_factor)

print("\n Loop Filter Configuration Parameters")
print(f"Sample Rate: {sample_rate}")
print(f"Loop Bandwidth: {loop_bandwidth}")
print(f"Damping Factor: {np.round(damping_factor, 5)}")
print(f"Loop Filter Coefficient K1: {np.round(k1, 5)}")
print(f"Loop Filter Coefficient K2: {np.round(k2, 5)}\n")


 Loop Filter Configuration Parameters
Sample Rate: 8
Loop Bandwidth: 0.16
Damping Factor: 0.70711
Loop Filter Coefficient K1: 0.05193
Loop Filter Coefficient K2: 0.00035

